In [1]:
import os
import sys
from pathlib import Path
import re
import jieba

In [2]:
# import shutil

In [3]:
# for path_item in SLR_wav_list:
#     wav_path = str(path_item)
#     shutil.move(wav_path,wav_path[0:-4]) 
 

## 准备词典

In [4]:
# !cd data && python ../tools/prep_dict.py lexicon.txt

In [5]:
# !./tools/prepare_dict.sh data

In [6]:
# !rm -rf gmm/data/dict/
# !mkdir gmm/data/dict/
# !cd data &&cp lexicon.txt silence_phones.txt nonsilence_phones.txt extra_questions.txt optional_silence.txt ../gmm/data/dict
# # !rm silence_phones.txt nonsilence_phones.txt extra_questions.txt optional_silence.txt

## SLR85生成wav.scp

In [7]:
SLR85_dir = "/home1/meichaoyang/Dataset/feats/SLR85/hifi"
SLR_wav_list=list(Path(SLR85_dir).rglob("*.wav"))

In [8]:
SLR_wav_list.sort(key=lambda Path: os.path.basename(os.path.splitext((str(Path)))[0]))

In [9]:
len(SLR_wav_list)

2719

In [10]:
with open('SLR_wav.scp', 'w') as f:
    for path_item in SLR_wav_list:
        wav_path = str(path_item)
        f.write(os.path.basename(os.path.splitext((wav_path))[0])+'\t'+wav_path+'\n')

### 并筛选一部分放入SLR_wav.scp.shuff文件中

In [12]:
# !bash tools/random_line.sh SLR_wav.scp 100 SLR_wav.scp.shuff
!cat SLR_wav.scp > SLR_wav.scp.shuff

> 并筛选一部分放入SLR_wav.scp.shuff文件中

## Aishell2数据处理

### 生成Aishell2的原始wav.scp

In [13]:
aishell_dir = "/home1/meichaoyang/Dataset/data_aishell2/data_aishell/wav/train"
aishell_wav_list=list(Path(aishell_dir).rglob("*.wav"))
aishell_wav_list.sort(key=lambda Path: os.path.basename(os.path.splitext((str(Path)))[0]))

In [14]:
aishell_scp_raw = {}
with open('Aishell_wav_raw.scp', 'w') as f:
    for path_item in aishell_wav_list:
        wav_path = str(path_item)
        aishell_scp_raw[os.path.basename(os.path.splitext((wav_path))[0])] = wav_path
        f.write(os.path.basename(os.path.splitext((wav_path))[0])+'\t'+wav_path+'\n')

### 对满足条件的标注进行筛选

In [16]:
pattern_utt = re.compile(r'/.*\.')
pattern_Eng = re.compile(u'[a-zA-Z\n]')

In [17]:
aishell_2_corp_raw = "/home1/meichaoyang/Dataset/data_aishell2/data_aishell/wav/trans.txt"
corp_map = {}
with open(aishell_2_corp_raw, "r") as f:
    for line in f:
        data = line.split()
        if pattern_Eng.search(data[1]) != None: ##删除小于10和非英文标注
            continue
        corp_map[pattern_utt.search(data[0]).group()[1:-1]] = data[1]

In [18]:
aishell_scp = {}
for i in sorted(aishell_scp_raw):
    if i not in corp_map.keys():
        continue
    aishell_scp[i] = aishell_scp_raw[i]

In [19]:
with open('aishell_2.scp', 'w') as f:
    for i in aishell_scp:
        f.write(i+'\t'+aishell_scp[i]+'\n')

In [20]:
with open('aishell_2_trans.txt', 'w') as f:
    for i in corp_map:
        f.write(i+'\t'+corp_map[i]+'\n')

### 筛选一部分到aishell_2.scp.shuff

In [21]:
# !bash tools/random_line.sh aishell_2.scp 2 aishell_2.scp.shuff
! cat aishell_2.scp > aishell_2.scp.shuff

## 合并数据

In [22]:
SLR_wav_scp = {}
SLR_wav_corp = {}
with open("SLR_wav.scp.shuff", "r") as f:
    for line in f:
        data = line.split()
        SLR_wav_scp[data[0]] = data[1]
        SLR_wav_corp[data[0]] = "NOISE你好NOISE米雅NOISE"

In [23]:
aishell_wav_scp = {}
with open("aishell_2.scp.shuff", "r") as f:
    for line in f:
        data = line.split()
        aishell_wav_scp[data[0]] = data[1]
        
aishell_wav_corp = {}       
with open("aishell_2_trans.txt", "r") as f:
    for line in f:
        data = line.split()
        if data[0] in aishell_wav_scp:
            aishell_wav_corp[data[0]] = data[1]

In [24]:
wav_scp = {**SLR_wav_scp,**aishell_wav_scp}
corpus = {**SLR_wav_corp, **aishell_wav_corp}

In [25]:
with open('wav.scp', 'w') as f:
    for i in sorted(wav_scp):
        f.write(i+'\t'+wav_scp[i]+'\n')

In [26]:
with open('corpus.txt', 'w') as f:
    for i in sorted(corpus):
        f.write(i+'\t'+corpus[i]+'\n')

### 生成词典并分词

In [27]:
!sed 's/\s.*$/ 1/' data/lexicon.txt > seg.dict

In [28]:
#!python seg_word/segmentword.py seg.dict corpus.txt corpus.seg oov_file




vocab_file="seg.dict"
trans_file="corpus.txt"
word_segmented_trans="corpus.seg"

jieba.set_dictionary(vocab_file)
with open(word_segmented_trans, "w") as f:
    for line in open(trans_file):
      key,trans = line.strip().split(None, 1)
      words = jieba.cut(trans, HMM=False) # turn off new word discovery (HMM-based)
      new_line = key + '\t' + " ".join(words)
      f.write(new_line + "\n")



Building prefix dict from /home1/meichaoyang/workspace/wake_up_align_44_1k/seg.dict ...
Dumping model to file cache /home1/meichaoyang/anaconda3/lib/python3.7/site-packages/jieba/cache/jieba.u64e3cf1df5ecc565728f91fad7ae6dd0.cache
Loading model cost 0.408 seconds.
Prefix dict has been built succesfully.


In [29]:
#!python tools/filter_scp.py corpus.seg wav.scp wav.scp.new

## utt2spk和spk2utt

In [30]:
new_wav_scp_path = "wav.scp"
new_wav_scp = {}
with open(new_wav_scp_path, "r") as f:
    for line in f:
        data = line.split()
        new_wav_scp[data[0]] = data[1]

In [31]:
pattern = re.compile(u'/[^/]+')
str1 = '/home/meichaoyang/dataset/SLR85/dev/SPEECHDATA/wav/SV0255/SV0255_2_00_F0026.wav'
pattern.findall(str1)[-2][1:]

'SV0255'

In [32]:
len(new_wav_scp)

895986

In [33]:
utts = list(new_wav_scp.items())

In [34]:
utt2spk = {}
for utt in utts:
    spk = pattern.findall(utt[1])[-2][1:]
    utt2spk[utt[0]] = spk

In [35]:
spk2utt = {}
for utt in utts:
    spk = pattern.findall(utt[1])[-2][1:]
    if spk not in spk2utt:
        spk2utt[spk] = []
    spk2utt[spk].append(utt[0])

In [36]:
with open('utt2spk', 'w') as f:
    for i in sorted(utt2spk):
        f.write(i+'\t'+utt2spk[i]+'\n')

In [37]:
len(utts)

895986

#### 根据utt2spk生成spk2utt

In [38]:
!gmm/utils/utt2spk_to_spk2utt.pl utt2spk > spk2utt

#### spk2utt，utt2spk，text(corpus.seg)，wav.scp(wav.scp.new)复制到train_{mfcc,fbank}

In [39]:
!rm -rf train_mfcc train_fbank
!mkdir train_mfcc train_fbank
!cp spk2utt utt2spk corpus.seg wav.scp train_mfcc

In [40]:
!mv train_mfcc/corpus.seg train_mfcc/text 
#!mv train_mfcc/wav.scp.new train_mfcc/wav.scp
!cd train_mfcc &&cp spk2utt utt2spk text wav.scp ../train_fbank

#### 特征提取`cd ext_feats && sh ext_{mfcc,fbank}.sh {train_fbank,train_mfcc,dev}`，需注意修改conf下mfcc，fbank参数

In [41]:
!cd ext_feats && sh ext_mfcc.sh ../train_mfcc && utils/fix_data_dir.sh ../train_mfcc

steps/make_mfcc.sh --nj 48 --cmd run.pl ../train_mfcc ../train_mfcc/log ../train_mfcc/_mfcc
utils/validate_data_dir.sh: Successfully validated data-directory ../train_mfcc
steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
Succeeded creating MFCC features for train_mfcc
steps/compute_cmvn_stats.sh ../train_mfcc ../train_mfcc/cmvn_log ../train_mfcc/_cmvn
Succeeded creating CMVN stats for train_mfcc
fix_data_dir.sh: kept all 895986 utterances.
fix_data_dir.sh: old files are kept in ../train_mfcc/.backup


In [42]:
!cd ext_feats && sh ext_fbank.sh ../train_fbank && utils/fix_data_dir.sh ../train_fbank

steps/make_fbank.sh --nj 48 --cmd run.pl ../train_fbank ../train_fbank/log ../train_fbank/_fbank
utils/validate_data_dir.sh: Successfully validated data-directory ../train_fbank
steps/make_fbank.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
Succeeded creating filterbank features for train_fbank
steps/compute_cmvn_stats.sh ../train_fbank ../train_fbank/cmvn_log ../train_fbank/_cmvn
Succeeded creating CMVN stats for train_fbank
fix_data_dir.sh: kept all 895986 utterances.
fix_data_dir.sh: old files are kept in ../train_fbank/.backup


In [43]:
!ln -s ../../train_mfcc gmm/data/train
!ln -s ../../train_fbank chain/data/train_fbank
!ln -s ../../gmm/data/train/ chain/data/train_mfcc

In [44]:
!cd gmm && make prepare_lang

utils/prepare_lang.sh --position-dependent-phones false `pwd`/data/dict "SPOKEN_NOISE" `pwd`/data/local/lang `pwd`/data/lang
utils/prepare_lang.sh --position-dependent-phones false /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/dict SPOKEN_NOISE /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/local/lang /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/lang
Checking /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/dict/silence_phones.txt ...
--> reading /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/dict/silence_phones.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/dict/silence_phones.txt is OK

Checking /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/dict/optional_silence.txt ...
--> reading /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/dict/optional_silence.txt
--> text seems to be UTF-8 or ASCII

--> /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/lang/L.fst is olabel sorted
--> /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/lang/L_disambig.fst is olabel sorted
--> SUCCESS [validating lang directory /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/lang]


In [ ]:
!cd gmm &&make -f Makefile gmm  | tee gmm.log

local/train_gmm.sh `pwd`/data/train `pwd`/data/cv
steps/train_mono.sh --boost-silence 1.25 --nj 24 --cmd run.pl /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/train data/lang exp/mono
steps/train_mono.sh: Initializing monophone system.
steps/train_mono.sh: Compiling training graphs
steps/train_mono.sh: Aligning data equally (pass 0)
steps/train_mono.sh: Pass 1
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 2
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 3
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 4
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 5
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 6
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 7
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 8
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 9
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 10
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pa

steps/align_si.sh --nj 24 --cmd run.pl --use-graphs true /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/train data/lang exp/tri2b exp/tri2b_ali
steps/align_si.sh: feature type is lda
steps/align_si.sh: aligning data in /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/train using model from exp/tri2b, putting alignments in exp/tri2b_ali
steps/diagnostic/analyze_alignments.sh --cmd run.pl data/lang exp/tri2b_ali
steps/diagnostic/analyze_alignments.sh: see stats in exp/tri2b_ali/log/analyze_alignments.log
steps/align_si.sh: done aligning data.
steps/train_sat.sh --cmd run.pl 15000 180000 /home1/meichaoyang/workspace/wake_up_align_44_1k/gmm/data/train data/lang exp/tri2b_ali exp/tri3b
steps/train_sat.sh: feature type is lda
steps/train_sat.sh: obtaining initial fMLLR transforms since not present in exp/tri2b_ali
steps/train_sat.sh: Accumulating tree stats
steps/train_sat.sh: Getting questions for tree clustering.
steps/train_sat.sh: Building the tree
steps/train_sat.sh:

In [ ]:
!export CUDA_VISIBLE_DEVICES=0,1,2 && cd chain && make chain |  tee chain.log

In [47]:
#!sed 's/[^\s]*\s//' corpus.seg > corpus_lm.txt
!ngram-count -text corpus_lm.txt -order 1 -unk -map-unk "<UNK>" -interpolate -lm corpus.lm

In [48]:
!cd chain/ && make make_graph

local/make_graph.sh /home1/meichaoyang/Workspace/wake_up/corpus.lm `pwd`/data/lang exp/chain/tdnn_attend `pwd`/data/dict
gzip: /home1/meichaoyang/Workspace/wake_up/corpus.lm: No such file or directory
Makefile:18: recipe for target 'make_graph' failed
make: *** [make_graph] Error 1


In [49]:
"asdf asdf".split(' ', 1 )

['asdf', 'asdf']

In [4]:
!du -sh chain/exp

23G	chain/exp
